<H1>Generalizing the Diet Problem</H1>

<H3>A Sample Instance</H3>

Recall that the objective of the diet problem is to find a combination of foods that meets some nutrient requirements. It is straightforward to write down a model for a small set of inputs such as the following.

<table>
<caption>Units of nutrients and cost per ounce</caption>
<tr>
<th> Food type </th> <th> Iron </th> <th> Calcium </th> <th> Cost </th>
</tr>
<tr>
<th> 1 </th> <th> 2 </th> <th> 0 </th> <th> 20 </th>
</tr>
<tr>
<th> 2 </th> <th> 0 </th> <th> 1 </th> <th> 10 </th>
</tr>
<tr>
<th> 3 </th> <th> 3 </th> <th> 2 </th> <th> 31 </th>
</tr>
<tr>
<th> 4 </th> <th> 1 </th> <th> 2 </th> <th> 11 </th>
</tr>
<tr>
<th> 5 </th> <th> 2 </th> <th> 1 </th> <th> 12 </th>
</tr>

Nutrient requirements: 21 units of iron and 12 units of calcium

We can simply define our decision variables $x_1,\ldots,x_5$ as the number of ounces to consume of each food type, and the resulting linear program is simply:

\begin{eqnarray}
\min_x && 20 x_1 + 10 x_2 + 31 x_3 + 11 x_4 + 12 x_5 \\
\mbox{s.t.} && 2 x_1 + 0 x_2 + 3 x_3 + 1 x_4 + 2 x_5 \ge 21 \nonumber \\
&& 0 x_1 + 1 x_2 + 2 x_3 + 2 x_4 + 1 x_5 \ge 12 \nonumber \\
&& x_1,\ldots,x_5 \ge 0.
\end{eqnarray}

Writing down this small LP is a useful exercise, but we will need to write down a more general version of the model if we are ever to write a program that could solve any instance of the diet problem.

<H3>Sets and Indices</H3>

An LP consists of decision variables, constraints, and an objective, all of which we'll need to define, but none of which we can define until we create a notation for the various sets in the problem. For the diet problem, the relevant entities are nutrients and food types. We'll start by defining the following sets:

* $i \in I$: nutrients
* $j \in J$: food types

The convention we prefer is to use an upper case letter to denote the full set, and a lower case letter to denote an element of that set. The symbol $\in$ can be read as ``in'', so $i \in I$ tells you that $i$ is a particular nutrient that is in $I$, the full set of nutrients.

Defining the relevant sets is typically the first step in modeling.

<H3>Data</H3>

Once we have defined our sets, we can take our input data and write it in a more general way. Since LPs can have any mixture of $\ge$, $\le$, and $=$ constraints, we can generalize the nutrient requirements to include both a lower and upper bound on each nutrient.

* $c_j$: per ounce cost of food type $j$
* $a_{ij}$: quantity of nutrient $i$ per ounce of food type $j$
* $l_i, u_i$: minimum, maximum daily requirements for nutrient $i$

<H3>Decision Variables</H3>

* $x_j$: the number of ounces to consume of food type $j$

With the decision variables and data written in a generic way, we can write down expressions for the total cost, and for the quantity of each nutrient in our diet.

<H3>Objective</H3>

The total cost can be obtained by multiplying the number of ounces consumed of a food type, $x_j$, by the per ounce cost of that food type, $c_j$, then summing over all food types $j \in J$. We'll use $\sum$ notation to denote sums, putting the set we are summing over under the $\sum$. The objective can therefore be written as $\sum_{j \in J} c_j x_j$.

<H3>Constraints</H3>

To write down constraints that enforce bounds on nutrient consumption, we'll need to write down an expression for the quantity of each nutrient we consume. 

Let's fix the nutrient $i$. Food type $j$'s contribution of nutrient $i$ will be the product of the per ounce quantity of nutrient $i$, $a_{ij}$, by the number of ounces consumed, $x_j$. We'll sum this product over all food types $j \in J$, again using $\sum$ notation. The resulting expression will be $\sum_{j \in J} a_{ij}x_j$.

This expression is valid for any nutrient $i \in I$. This gives us all we need to formulate the diet problem as an LP.

<H3>Formulation</H3>
\begin{eqnarray}
\min_x && \sum_{j \in J} c_j x_j \nonumber \\
\mbox{s.t.} && l_i \le \sum_{j \in J} a_{ij} x_j \le u_i,\;\;i \in I \nonumber \\
&& x_j \ge 0,\;\;j \in J. \nonumber
\end{eqnarray}

We can now write a method to build the model and solve it with Gurobi, but let's define our inputs and write a test case first. We can test with a sample instance that we already know the solution to.

In [ ]:
import unittest
import gurobipy as grb
GRB = grb.GRB
class TestDiet(unittest.TestCase):
    def test_diet(self):
        costs = {1: 20, 2: 10, 3: 31, 4: 11, 5: 12}
        nutrient_densities = {(1, 'iron'): 2, (1, 'calcium'): 0,
                              (2, 'iron'): 0, (2, 'calcium'): 1,
                              (3, 'iron'): 3, (3, 'calcium'): 2,
                              (4, 'iron'): 1, (4, 'calcium'): 2,
                              (5, 'iron'): 2, (5, 'calcium'): 1}
        nutrient_requirements = {'iron': [21, GRB.INFINITY], 'calcium': [12, GRB.INFINITY]}
        ounces_consumed = solve_diet_problem(nutrient_densities, costs, nutrient_requirements)
        self.assertAlmostEqual(ounces_consumed[1], 0)
        self.assertAlmostEqual(ounces_consumed[2], 0)
        self.assertAlmostEqual(ounces_consumed[3], 0)
        self.assertAlmostEqual(ounces_consumed[4], 1)
        self.assertAlmostEqual(ounces_consumed[5], 10)

Now let's implement the solve_diet_problem method. We have a constraint that puts both a lower and upper bound on a linear expression. We'll find the Model.addRange method useful for this, and prefer to use that as opposed to making two calls to Model.addConstr.

In [ ]:
grb.Model.addRange?

In [ ]:
def solve_diet_problem(nutrient_densities, costs, nutrient_requirements):
    m = grb.Model()
    ounces_consumed = {food_type: m.addVar(obj=cost, name='ounces_consumed.' + str(food_type))
                       for food_type, cost in costs.iteritems()}
    m.update()
    nutrient_constraints = {}
    food_types = costs.keys()
    for nutrient, (min_requirement, max_requirement) in nutrient_requirements.iteritems():
        nutrient_consumed = grb.quicksum(nutrient_densities[food_type, nutrient]*ounces_consumed[food_type]
                                         for food_type in food_types)
        constr = m.addRange(nutrient_consumed, min_requirement, max_requirement,
                           'nutrient.' + str(nutrient))
        nutrient_constraints[nutrient] = constr
    m.optimize()
    if m.status == GRB.OPTIMAL:
        return {food_type: var.X for food_type, var in ounces_consumed.iteritems()}
    raise Exception("Model was infeasible.")

In [ ]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestDiet)
unittest.TextTestRunner().run(suite)